# Analyzing INTERNET speeds in Morocco

In [4]:

from datetime import datetime

import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import requests
import re

from tqdm.notebook import tqdm  # progress bar in Jupyter
from datetime import datetime
from shapely.geometry import Point
from adjustText import adjust_text

---

## Download data

First, download OOKLA data

In [2]:
def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")
    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)

def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")
    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)

def quarter_end(year: int, q: int) -> datetime:
    if q == 4:
        return datetime(year + 1, 1, 1)
    return quarter_start(year, q + 1)

def get_tile_url(service_type: str, year: int, q: int) -> str | None:
    dt = quarter_start(year, q)
    end_dt = quarter_end(year, q)
    now = datetime.utcnow()

    if now < end_dt:
        print(f"⏩ Skipping {service_type} {year} Q{q} (quarter not yet complete)")
        return None

    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_{service_type}_tiles.zip"
    return url

def download_file_with_progress(url: str, output_dir: str = "data") -> str:
    os.makedirs(output_dir, exist_ok=True)
    
    local_filename = os.path.join(output_dir, url.split("/")[-1])
    
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte

    t = tqdm(total=total_size, unit='iB', unit_scale=True, desc=f"Downloading {os.path.basename(local_filename)}")

    with open(local_filename, 'wb') as f:
        for data in response.iter_content(block_size):
            t.update(len(data))
            f.write(data)

    t.close()

    if total_size != 0 and t.n != total_size:
        print("⚠️ WARNING: Download might be incomplete.")
    
    return local_filename

configure ctype,years,and quarters as you need

In [3]:
ctype = ["fixed","mobile"]
years = [2023,2025]
quarters = [1, 2, 3, 4]
for t in ctype :
    for year in years :
        for q in quarters :
            url = get_tile_url(t, year, q)
           # download_file_with_progress(url) #uncomment to download

⏩ Skipping fixed 2025 Q2 (quarter not yet complete)
⏩ Skipping fixed 2025 Q3 (quarter not yet complete)
⏩ Skipping fixed 2025 Q4 (quarter not yet complete)
⏩ Skipping mobile 2025 Q2 (quarter not yet complete)
⏩ Skipping mobile 2025 Q3 (quarter not yet complete)
⏩ Skipping mobile 2025 Q4 (quarter not yet complete)


/tmp/ipykernel_145205/3529908371.py:21: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()


In [4]:
Morocco = gp.read_file("data/Morocco/morocco.shp")

In [5]:

# List all matching zip files
zip_files = [f for f in os.listdir("data") if f.endswith(".zip") and "tiles" in f]

# Progress bar for loading
for filename in tqdm(zip_files, desc="Loading tiles"):
    path = os.path.join("data", filename)
    try:
        gdf = gp.read_file(path)
        print(f"LOADED : {filename}: {len(gdf)} features")
        name = filename
        gdf = gdf.to_crs(Morocco.crs)
        morocco_tiles = gp.clip(gdf, Morocco)
        output_name = name.replace(".zip", "_morocco.shp")
        output_path = os.path.join("MoroccoData", output_name)
        morocco_tiles.to_file(output_path, driver="ESRI Shapefile")
        print(f"SAVED: {output_path} ({len(morocco_tiles)} features)")
    except Exception as e:
        print(f"❌ Failed to load {filename}: {e}")
    

📥 Loading tiles:   0%|          | 0/10 [00:00<?, ?it/s]

✅ 2025-01-01_performance_fixed_tiles.zip: 6364159 features | CRS: EPSG:4326 | Columns: ['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices', 'geometry']
✅ Saved: MoroccoData/2025-01-01_performance_fixed_tiles_morocco.shp (18874 features)
✅ 2024-07-01_performance_mobile_tiles.zip: 3773658 features | CRS: EPSG:4326 | Columns: ['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices', 'geometry']
✅ Saved: MoroccoData/2024-07-01_performance_mobile_tiles_morocco.shp (12335 features)
✅ 2024-01-01_performance_mobile_tiles.zip: 3674000 features | CRS: EPSG:4326 | Columns: ['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices', 'geometry']
✅ Saved: MoroccoData/2024-01-01_performance_mobile_tiles_morocco.shp (12563 features)
✅ 2025-01-01_performance_mobile_tiles.zip: 3388115 features | CRS: EPSG:4326 | Columns: ['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices', 'geometry']
✅ Saved: MoroccoData/2025-01-01_performance_mobi

extracted from link[ https://github.com/teamookla/ookla-open-data/edit/master/README.md ]

#### Tile Attributes
Each tile contains the following adjoining attributes:

| Field Name        | Type        | Description                                                                                                                             | Notes                                                                                                                                                                              |
|-------------------|-------------|-----------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `avg_d_kbps`      | Integer     | The average download speed of all tests performed in the tile, represented in kilobits per second.                                      |                                                                                                                                                                                    |
| `avg_u_kbps`      | Integer     | The average upload speed of all tests performed in the tile, represented in kilobits per second.                                        |                                                                                                                                                                                    |
| `avg_lat_ms`      | Integer     | The average latency of all tests performed in the tile, represented in milliseconds                                                     |                                                                                                                                                                                    |
| `avg_lat_down_ms` | Integer     | The average latency under load of all tests performed in the tile as measured during the download phase of the test. Represented in ms. | Parquet-only. Added 2023-02-23 beginning in Q4 2022 dataset. This column is sparsely populated-- some rows will have a null value as not all versions of Speedtest can perform this measurement. |
| `avg_lat_up_ms`   | Integer     | The average latency under load of all tests performed in the tile as measured during the upload phase of the test. Represented in ms.   | Parquet-only. Added 2023-02-23 beginning in Q4 2022 dataset. This column is sparsely populated-- some rows will have a null value as not all versions of Speedtest can perform this measurement. |
| `tests`           | Integer     | The number of tests taken in the tile. |
| `devices`         | Integer     | The number of unique devices contributing tests in the tile. |
| `quadkey`         | Text        | The quadkey representing the tile.  |
| `tile_x`			| Numeric	  | X coordinate of the tile's centroid.| Added 2023-07-01 beginning in the Q3 2023 dataset.
| `tile_y`          | Numeric     | Y coordinate of the tile's centroid.| Added 2023-07-01 beginning in the Q3 2023 dataset.


#### Quadkeys

[Quadkeys](https://docs.microsoft.com/en-us/bingmaps/articles/bing-maps-tile-system) can act as a unique identifier for the tile. This can be useful for joining data spatially from multiple periods (quarters), creating coarser spatial aggregations without using geospatial functions, spatial indexing, partitioning, and an alternative for storing and deriving the tile geometry.

#### Layers
Two layers are distributed as separate sets of files:

* `performance_mobile_tiles` - Tiles containing tests taken from mobile devices with GPS-quality location and a cellular connection type (e.g. 4G LTE, 5G NR).
* `performance_fixed_tiles` - Tiles containing tests taken from mobile devices with GPS-quality location and a non-cellular connection type (e.g. WiFi, ethernet).

#### Time Period and Update Frequency

Layers are generated based on a quarter year of data (three months) and files will be updated and added on a quarterly basis. A `/year=2020/quarter=1/` period, the first quarter of the year 2020, would include all data generated on or after `2020-01-01` and before `2020-04-01`.

Data is subject to be reaggregated regularly in order to honor Data Subject Access Requests (DSAR) as is applicable in certain jurisdictions under laws including but not limited to General Data Protection Regulation (GDPR), California Consumer Privacy Act (CCPA), and Lei Geral de Proteção de Dados (LGPD). Therefore, data accessed at different times may result in variation in the total number of tests, tiles, and resulting performance metrics.



## LOAD MOROCCO DATA

In [2]:

DATA_FOLDER = "MoroccoData"

# Grab only shapefiles that end with "_morocco.shp"
Morocco_Data_files = [
    f for f in os.listdir(DATA_FOLDER) 
    if f.endswith("_morocco.shp")
]

# This will accumulate your “wide” data
final_gdf = None

# Regex to capture:
#   2024-01-01_performance_fixed_tiles_morocco.shp
#   group(1) => 2024-01-01
#   group(2) => fixed or mobile
filename_pattern = re.compile(r'(\d{4}-\d{2}-\d{2})_.*?(fixed|mobile).*?_morocco\.shp')

for filename in tqdm(Morocco_Data_files, desc="Loading Morocco tiles"):
    path = os.path.join(DATA_FOLDER, filename)
    try:
        gdf = gp.read_file(path)
        print(f"LOADED : {filename}: {len(gdf)} features")

        # Extract quarter/date and type from filename
        match = filename_pattern.search(filename)
        if not match:
            print(f"❌ Could not parse quarter/type from {filename}")
            continue
        
        quarter_str = match.group(1)  # e.g. "2024-01-01"
        conn_type  = match.group(2)   # e.g. "fixed" or "mobile"
        
        # Build a rename map. We keep 'quadkey' and 'geometry' the same
        rename_map = {}
        for col in gdf.columns:
            if col not in ['quadkey', 'geometry']:
                # Prepend <quarter>_<type>_
                new_col = f"{quarter_str}_{conn_type}_{col}"
                rename_map[col] = new_col
        
        # Rename columns
        gdf_renamed = gdf.rename(columns=rename_map)
        
        # If final_gdf is None, just store this one.
        if final_gdf is None:
            final_gdf = gdf_renamed
        else:
            # We'll do a full outer merge only on 'quadkey'
            # Because tile_x & tile_y do not exist in your shapefiles
            gdf_no_geom = gdf_renamed.drop(columns=['geometry'])
            final_gdf = final_gdf.merge(
                gdf_no_geom,
                on='quadkey',
                how='outer'
            )
    except Exception as e:
        print(f"❌ Failed to load {filename}: {e}")

# At this point, final_gdf is your “wide” GeoDataFrame with columns like:
#   quadkey, geometry, 2024-01-01_fixed_avg_d_kbps, 2024-01-01_fixed_avg_u_kbps, etc.

# Save as a GeoPackage instead of ESRI Shapefile so you don’t lose column names:
final_out = "all_quarters_combined.gpkg"
final_gdf.to_file(final_out, driver="GPKG")
print(f"✅ Saved combined dataset to: {final_out}")


# -----------------------------------------
# Create a second file that has differences 
# between consecutive quarters for each type
import pandas as pd

diff_gdf = final_gdf.copy()

# We’ll parse all columns that match e.g. 2024-01-01_fixed_avg_d_kbps
col_pattern = re.compile(r'(\d{4}-\d{2}-\d{2})_(fixed|mobile)_(.*)')

# Build a structure to group columns by (type, metric), e.g. (fixed, avg_d_kbps).
quarter_type_to_metrics = {}

for col in diff_gdf.columns:
    m = col_pattern.match(col)
    if m:
        q_str = m.group(1)
        t_str = m.group(2)
        metric_name = m.group(3)
        
        # Keep track of columns by quarter
        quarter_type_to_metrics.setdefault((t_str, metric_name), []).append(q_str)

# For each (type, metric), sort quarters & build difference columns
for (conn_type, metric_name), quarter_list in quarter_type_to_metrics.items():
    quarter_list_sorted = sorted(quarter_list)
    for i in range(1, len(quarter_list_sorted)):
        prev_q = quarter_list_sorted[i-1]
        curr_q = quarter_list_sorted[i]
        
        prev_col = f"{prev_q}_{conn_type}_{metric_name}"
        curr_col = f"{curr_q}_{conn_type}_{metric_name}"
        
        # If for some reason we’re missing a column, skip
        if prev_col not in diff_gdf.columns or curr_col not in diff_gdf.columns or metric_name in ['tests', 'devices']:
            continue
        
        diff_col_name = f"D_{curr_q}_M_{prev_q}_{conn_type}_{metric_name}"
        diff_gdf[diff_col_name] = diff_gdf[curr_col] - diff_gdf[prev_col]

# Save differences as well, again in a GeoPackage
diff_out = "all_quarters_differences.gpkg"
diff_gdf.to_file(diff_out, driver="GPKG")
print(f"✅ Saved differences dataset to: {diff_out}")

Loading Morocco tiles:   0%|          | 0/10 [00:00<?, ?it/s]

LOADED : 2024-04-01_performance_fixed_tiles_morocco.shp: 18484 features
LOADED : 2024-07-01_performance_fixed_tiles_morocco.shp: 18587 features
LOADED : 2024-10-01_performance_fixed_tiles_morocco.shp: 18414 features
LOADED : 2024-04-01_performance_mobile_tiles_morocco.shp: 13523 features
LOADED : 2024-01-01_performance_fixed_tiles_morocco.shp: 19127 features
LOADED : 2024-07-01_performance_mobile_tiles_morocco.shp: 12335 features
LOADED : 2025-01-01_performance_fixed_tiles_morocco.shp: 18874 features
LOADED : 2024-10-01_performance_mobile_tiles_morocco.shp: 12340 features
LOADED : 2025-01-01_performance_mobile_tiles_morocco.shp: 12002 features
LOADED : 2024-01-01_performance_mobile_tiles_morocco.shp: 12563 features
✅ Saved combined dataset to: all_quarters_combined.gpkg
✅ Saved differences dataset to: all_quarters_differences.gpkg


In [ ]:
import os
import geopandas as gpd

# Vérifions le contenu du dossier
data_dir = os.path.join(os.getcwd(), "data")
print("Sous-dossiers de data:", os.listdir(data_dir))


communes_dir = os.path.join(data_dir, "density_population_comune")
print("Fichiers dans density_population_comune:", os.listdir(communes_dir))

# Chemin exact vers le .shp (ajustez selon le nom retourné ci-dessus)
shp_path = os.path.join(communes_dir, "populaion_commune.shp")

# Optionnel : reconstruire le .shx si besoin
os.environ["SHAPE_RESTORE_SHX"] = "YES"

# Chargement GeoPandas
print("Lecture de :", shp_path)
communes = gpd.read_file(shp_path)

print("✅ Chargé :", communes.shape, "lignes")
print("CRS :", communes.crs)
print("Colonnes :", communes.columns.tolist())


Sous-dossiers de data: ['density_population_comune', 'Morocco', 'Morocco-Counties-shapefile']
Fichiers dans density_population_comune: ['populaion_commune.cpg', 'populaion_commune.dbf', 'populaion_commune.prj', 'populaion_commune.shp', 'populaion_commune.shx']
Lecture de : c:\Users\Ahmed\OoklaOpenDataMorocco\data\density_population_comune\populaion_commune.shp
✅ Chargé : (1505, 10) lignes
CRS : EPSG:4326
Colonnes : ['Code_Commu', 'Code_Provi', 'CODE_REGIO', 'nom', 'Marocains_', 'Etrangers_', 'Populati_1', 'Menages_', 'nom_arabe', 'geometry']


In [14]:
import os

# 1. Confirmez le répertoire courant
print("Working dir :", os.getcwd())

# 2. Cherchez récursivement les .gpkg
for root, dirs, files in os.walk(os.getcwd()):
    for f in files:
        if f.endswith(".gpkg"):
            print("→ Trouvé :", f, "dans", root)


Working dir : c:\Users\Ahmed\OoklaOpenDataMorocco


In [15]:
import os
import geopandas as gpd

# 1) Construire le chemin vers le GeoPackage
base_dir   = os.getcwd()  # c:\Users\Ahmed\OoklaOpenDataMorocco
data_dir   = os.path.join(base_dir, "data", "density_population_comune")
gpkg_path  = os.path.join(data_dir, "all_quarters_differences.gpkg")

print("Lecture de :", gpkg_path)
tiles = gpd.read_file(gpkg_path)

print("✅ Chargé :", tiles.shape, "lignes")
print("Colonnes :", tiles.columns.tolist())


Lecture de : c:\Users\Ahmed\OoklaOpenDataMorocco\data\density_population_comune\all_quarters_differences.gpkg
✅ Chargé : (42997, 76) lignes
Colonnes : ['quadkey', '2024-04-01_fixed_avg_d_kbps', '2024-04-01_fixed_avg_u_kbps', '2024-04-01_fixed_avg_lat_ms', '2024-04-01_fixed_tests', '2024-04-01_fixed_devices', '2024-07-01_fixed_avg_d_kbps', '2024-07-01_fixed_avg_u_kbps', '2024-07-01_fixed_avg_lat_ms', '2024-07-01_fixed_tests', '2024-07-01_fixed_devices', '2024-10-01_fixed_avg_d_kbps', '2024-10-01_fixed_avg_u_kbps', '2024-10-01_fixed_avg_lat_ms', '2024-10-01_fixed_tests', '2024-10-01_fixed_devices', '2024-04-01_mobile_avg_d_kbps', '2024-04-01_mobile_avg_u_kbps', '2024-04-01_mobile_avg_lat_ms', '2024-04-01_mobile_tests', '2024-04-01_mobile_devices', '2024-01-01_fixed_avg_d_kbps', '2024-01-01_fixed_avg_u_kbps', '2024-01-01_fixed_avg_lat_ms', '2024-01-01_fixed_tests', '2024-01-01_fixed_devices', '2024-07-01_mobile_avg_d_kbps', '2024-07-01_mobile_avg_u_kbps', '2024-07-01_mobile_avg_lat_ms', '

In [17]:
import os
import geopandas as gpd
import pandas as pd

# 1) Chargez la table des tuiles
tiles = gpd.read_file(
    os.path.join("data", "density_population_comune", "all_quarters_differences.gpkg")
)

# 2) Chargez la couche communes et calculez la densité
communes = gpd.read_file(
    os.path.join("data", "density_population_comune", "populaion_commune.shp")
)

# (a) Passez en métrique pour la surface
communes_m = communes.to_crs(epsg=3857)
communes["area_km2"] = communes_m.geometry.area / 1e6

# (b) Calculez densité : population / km²
communes["densite_pop"] = communes["Populati_1"] / communes["area_km2"]

# (c) Remettez en EPSG:4326 pour matcher les tuiles
communes = communes.to_crs(tiles.crs)

# 3) Jointure spatiale « within »  : chaque tuile hérite de la densité de sa commune
tiles_with_pop = gpd.sjoin(
    tiles,
    communes[["geometry", "densite_pop"]],
    how="left",
    predicate="within"
)

# 4) (Optionnel) Convertir en DataFrame pandas pour exporter sans géométrie
df_model = pd.DataFrame(tiles_with_pop.drop(columns="geometry"))

# 5) Sauvegarder votre jeu de données enrichi
#  - géopackage si vous voulez garder la géométrie
tiles_with_pop.to_file(
    os.path.join("data", "tiles_with_density.gpkg"),
    driver="GPKG"
)

#  - ou CSV si vous ne voulez que les variables tabulaires
df_model.to_csv(
    os.path.join("data", "tiles_with_density.csv"),
    index=False
)

print("✅ Jeu de données enrichi enregistré :")
print("  • GeoPackage → data/tiles_with_density.gpkg")
print("  • CSV         → data/tiles_with_density.csv")


✅ Jeu de données enrichi enregistré :
  • GeoPackage → data/tiles_with_density.gpkg
  • CSV         → data/tiles_with_density.csv


In [19]:
import os
import geopandas as gpd
import numpy as np
from sklearn.ensemble       import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics        import mean_squared_error

# 1️⃣ INPUT
# GeoPackage enrichi avec géométrie et densité
in_path = "data/tiles_with_density.gpkg"

# 2️⃣ CHARGEMENT
tiles = gpd.read_file(in_path)

# 3️⃣ DÉFINITION DES FEATURES ET DE LA TARGET
features = [
    "2024-10-01_mobile_avg_d_kbps",               # débit mobile dernier trimestre
    "2024-10-01_fixed_avg_d_kbps",                # débit fixe dernier trimestre
    "D_2024-10-01_M_2024-07-01_mobile_avg_d_kbps", # variation mobile Q-1 vs Q-2
    "densite_pop"                                 # densité de population
]
target = "2025-01-01_mobile_avg_d_kbps"           # débit mobile à prédire

# 4️⃣ PRÉPARATION DU JEU D’ENTRAÎNEMENT
df = tiles[features + [target]].dropna()
X, y = df[features], df[target]

# 5️⃣ SPLIT TRAIN / TEST
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 6️⃣ ENTRAÎNEMENT DU RANDOM FOREST
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# 7️⃣ ÉVALUATION
y_pred = model.predict(X_test)
mse  = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"📊 RMSE (test) = {rmse:.2f} kbps")

# 8️⃣ SCORING DE TOUTES LES TUILES
#  - pred_rf   : débit prédit
#  - residu_rf : écart observé − prédit
#  - score_rf  : priorité = densité × (−résidu)
tiles["pred_rf"]   = model.predict(tiles[features].fillna(0))
tiles["residu_rf"] = tiles[target] - tiles["pred_rf"]
tiles["score_rf"]  = - tiles["residu_rf"] * tiles["densite_pop"]

# 9️⃣ EXPORT DES RÉSULTATS
out_path = "data/tiles_score_rf.gpkg"
tiles.to_file(out_path, driver="GPKG")
print(f"✅ Résultats sauvegardés dans {out_path}")


📊 RMSE (test) = 49070.83 kbps
✅ Résultats sauvegardés dans data/tiles_score_rf.gpkg


In [25]:
# 0️⃣ (Jupyter uniquement) Installer mgwr si besoin :
# !pip install mgwr

import numpy as np
import geopandas as gpd
from mgwr.gwr    import GWR
from mgwr.sel_bw import Sel_BW

# 1️⃣ INPUT
in_path = "data/tiles_with_density.gpkg"
tiles   = gpd.read_file(in_path)  # CRS EPSG:4326

# 2️⃣ PROJECTION en métrique
tiles_proj = tiles.to_crs(epsg=3857)

# 3️⃣ DÉFINITIONS
target    = "2025-01-01_mobile_avg_d_kbps"
covar_den = "densite_pop"
covar_dl1 = "2024-10-01_mobile_avg_d_kbps"

# 4️⃣ FILTRAGE NaN
cols_needed = [covar_den, covar_dl1, target]
tiles_clean = tiles_proj.dropna(subset=cols_needed)

# 5️⃣ CENTROÏDES + COORDS
centroids = tiles_clean.geometry.centroid
coords    = np.vstack((centroids.x, centroids.y)).T   # shape (n,2)

# 6️⃣ MATRICES X et y
X = tiles_clean[[covar_den, covar_dl1]].values         # (n,2)
y = tiles_clean[[target]].values                       # (n,1)

# 7️⃣ RECHERCHE DE LA BANDE PASSANTE
selector  = Sel_BW(coords, y, X)
bandwidth = selector.search()
print(f"🏎️ Bandwidth retenue : {bandwidth:.2f}")

# 8️⃣ FIT GWR
gwr_model   = GWR(coords, y, X, bandwidth)
gwr_results = gwr_model.fit()

# 9️⃣ PREDICTION EXPLICITE
pred_res = gwr_model.predict(coords, X)
tiles_clean["pred_gwr"]   = pred_res.predictions.flatten()
tiles_clean["residu_gwr"] = tiles_clean[target] - tiles_clean["pred_gwr"]

# 🔟 SCORE DE PRIORITÉ
tiles_clean["score_gwr"] = - tiles_clean["residu_gwr"] * tiles_clean[covar_den]

# 1️⃣1️⃣ REINJECTION dans le GeoDataFrame complet
for col in ["pred_gwr", "residu_gwr", "score_gwr"]:
    tiles_proj.loc[tiles_clean.index, col] = tiles_clean[col]

# 1️⃣2️⃣ REMISE EN CRS ORIGINE
result = tiles_proj.to_crs(tiles.crs)

# 1️⃣3️⃣ EXPORT
out_path = "data/tiles_score_gwr.gpkg"
result.to_file(out_path, driver="GPKG")
print(f"✅ GeoPackage GWR enregistré dans : {out_path}")


🏎️ Bandwidth retenue : 230.00


C:\Users\Ahmed\AppData\Roaming\Python\Python312\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Ahmed\AppData\Roaming\Python\Python312\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Ahmed\AppData\Roaming\Python\Python312\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

✅ GeoPackage GWR enregistré dans : data/tiles_score_gwr.gpkg


In [27]:
# 1️⃣ Contexte & dépendances
#   * hdbscan pour le clustering, sklearn pour la normalisation
import os
import geopandas as gpd
from sklearn.preprocessing import StandardScaler
import hdbscan

# 2️⃣ Inputs
#   - Toujours “tiles_with_density.gpkg”
#   - On clusterise sur :
#       • 2024-10-01_mobile_avg_d_kbps
#       • D_2024-10-01_M_2024-07-01_mobile_avg_d_kbps
#       • densite_pop
tiles = gpd.read_file("data/tiles_with_density.gpkg")

feat = tiles[[
    "2024-10-01_mobile_avg_d_kbps",
    "D_2024-10-01_M_2024-07-01_mobile_avg_d_kbps",
    "densite_pop"
]].fillna(0)

# 3️⃣ Étapes clés
# 3.1 Standardisation
Xs = StandardScaler().fit_transform(feat)

# 3.2 Clustering HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=50)
labels    = clusterer.fit_predict(Xs)
tiles["cluster"] = labels

# 3.3 Définir les clusters critiques
#    On considère critiques ceux dont la densité est dans le top 25% et débit dans le bottom 25%
summary = tiles.groupby("cluster").agg({
    "densite_pop": "mean",
    "2024-10-01_mobile_avg_d_kbps": "mean"
})
dens_high = summary["densite_pop"].quantile(0.75)
dl_low    = summary["2024-10-01_mobile_avg_d_kbps"].quantile(0.25)
top_clusters = summary[
    (summary["densite_pop"] >= dens_high) &
    (summary["2024-10-01_mobile_avg_d_kbps"] <= dl_low)
].index.tolist()

# 3.4 Score binaire : 1 si cluster critique, 0 sinon
tiles["score_hdb"] = tiles["cluster"].apply(lambda c: 1 if c in top_clusters else 0)

# 4️⃣ Outputs
#   - Dans `tiles` :
#       • cluster    : label HDBSCAN
#       • score_hdb  : indicateur de zone critique
tiles.to_file("data/tiles_score_hdb.gpkg", driver="GPKG")
print("➡️ résultats sauvegardés dans data/tiles_score_hdb.gpkg")


C:\Users\Ahmed\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\Ahmed\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


➡️ résultats sauvegardés dans data/tiles_score_hdb.gpkg
